In [2]:
# import calibration_tools
import os
import torchvision.datasets as dset
import torchvision.transforms as trn
import torchvision.transforms.functional as trnF
import torchvision.models as models
import torch.utils.model_zoo as model_zoo
from torchvision import models
from __future__ import print_function
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import torchvision
import numpy as np
import time
from scipy import misc
import pretrainedmodels

In [3]:
ROOT = "/scratch/as14770/CV-FinalProject"

PATH_CONFIDENCE_BASE_IN = ROOT + "/confidence_Base_In.txt"
PATH_CONFIDENCE_BASE_OUT = ROOT + "/confidence_Base_Out.txt"
PATH_CONFIDENCE_OUR_IN = ROOT + "/confidence_Our_In.txt"
PATH_CONFIDENCE_OUR_OUT = ROOT + "/confidence_Our_Out.txt"


epsilon = 0 # perturbation 21 evenly spaced numbers b/w 0 and 0.004 
temperature = 1000  # temperature scaling [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]
criterion = nn.CrossEntropyLoss()
CUDA_DEVICE = 0   # set to 1 to use gpu
# TODO: start and end also may need to be tuned

In [4]:
# def tpr95(name):
#     #calculate the falsepositive error when tpr is 95%
#     # calculate baseline
# #     T = 1
#     indistr = np.loadtxt(PATH_CONFIDENCE_BASE_IN, delimiter=',')
#     outdistr = np.loadtxt(PATH_CONFIDENCE_BASE_OUT, delimiter=',')
# #     if name == "CIFAR-10": 
# #       start = 0.1
# #       end = 1 
# #     if name == "CIFAR-100": 
#     start = 0.001
#     end = 1 
#     gap = (end- start)/100000
#     #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
#     Y1 = outdistr[:, 2]
#     X1 = indistr[:, 2]
#     total = 0.0
#     fpr = 0.0
#     for delta in np.arange(start, end, gap):
#         tpr = np.sum(np.sum(X1 >= delta)) / np.float(len(X1))
#         error2 = np.sum(np.sum(Y1 > delta)) / np.float(len(Y1))
#         if tpr <= 0.9505 and tpr >= 0.9495:
#             fpr += error2
#             total += 1
#     fprBase = fpr/total

#     # calculate our algorithm
# #     T = 1000
#     indistr = np.loadtxt(PATH_CONFIDENCE_OUR_IN, delimiter=',')
#     outdistr = np.loadtxt(PATH_CONFIDENCE_OUR_OUT, delimiter=',')
# #     if name == "CIFAR-10": 
# #       start = 0.1
# #       end = 0.12 
# #     if name == "CIFAR-100": 
# #       start = 0.01
# #       end = 0.0104  
#     start = 0.001
#     end = 0.0011
#     gap = (end- start)/100000
#     #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
#     Y1 = outdistr[:, 2]
#     X1 = indistr[:, 2]
#     total = 0.0
#     fpr = 0.0
#     for delta in np.arange(start, end, gap):
#         tpr = np.sum(np.sum(X1 >= delta)) / np.float(len(X1))
#         error2 = np.sum(np.sum(Y1 > delta)) / np.float(len(Y1))
#         if tpr <= 0.9505 and tpr >= 0.9495:
#             fpr += error2
#             total += 1
#     fprNew = fpr/total
            
#     return fprBase, fprNew

# def auroc(name):
#     #calculate the AUROC
#     # calculate baseline
# #     T = 1
#     indistr = np.loadtxt(PATH_CONFIDENCE_BASE_IN, delimiter=',')
#     outdistr = np.loadtxt(PATH_CONFIDENCE_BASE_OUT, delimiter=',')
# #     if name == "CIFAR-10": 
# #       start = 0.1
# #       end = 1 
# #     if name == "CIFAR-100": 
#     start = 0.001
#     end = 1 
#     gap = (end- start)/100000
#     #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
#     Y1 = indistr[:, 2]
#     X1 = outdistr[:, 2]
#     aurocBase = 0.0
#     fprTemp = 1.0
#     for delta in np.arange(start, end, gap):
#         tpr = np.sum(np.sum(X1 >= delta)) / np.float(len(X1))
#         fpr = np.sum(np.sum(Y1 > delta)) / np.float(len(Y1))
#         aurocBase += (-fpr+fprTemp)*tpr
#         fprTemp = fpr
#     aurocBase += fpr * tpr
#     # calculate our algorithm
# #     T = 1000
#     indistr = np.loadtxt(PATH_CONFIDENCE_OUR_IN, delimiter=',')
#     outdistr = np.loadtxt(PATH_CONFIDENCE_OUR_OUT, delimiter=',')
# #     if name == "CIFAR-10": 
# #       start = 0.1
# #       end = 0.12 
# #     if name == "CIFAR-100": 
# #       start = 0.01
# #       end = 0.0104   
#     start = 0.001
#     end = 0.0011
#     gap = (end- start)/100000
#     #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
#     Y1 = outdistr[:, 2]
#     X1 = indistr[:, 2]
#     aurocNew = 0.0
#     fprTemp = 1.0
#     for delta in np.arange(start, end, gap):
#         tpr = np.sum(np.sum(X1 >= delta)) / np.float(len(X1))
#         fpr = np.sum(np.sum(Y1 >= delta)) / np.float(len(Y1))
#         aurocNew += (-fpr+fprTemp)*tpr
#         fprTemp = fpr
#     aurocNew += fpr * tpr
#     return aurocBase, aurocNew

# def auprIn(name):
#     #calculate the AUPR
#     # calculate baseline
# #     T = 1
#     indistr = np.loadtxt(PATH_CONFIDENCE_BASE_IN, delimiter=',')
#     outdistr = np.loadtxt(PATH_CONFIDENCE_BASE_OUT, delimiter=',')
# #     if name == "CIFAR-10": 
# #       start = 0.1
# #       end = 1 
# #     if name == "CIFAR-100": 
#     start = 0.001
#     end = 1   
#     gap = (end- start)/100000
#     precisionVec = []
#     recallVec = []
#         #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
#     Y1 = outdistr[:, 2]
#     X1 = indistr[:, 2]
#     auprBase = 0.0
#     recallTemp = 1.0
#     for delta in np.arange(start, end, gap):
#         tp = np.sum(np.sum(X1 >= delta)) / np.float(len(X1))
#         fp = np.sum(np.sum(Y1 >= delta)) / np.float(len(Y1))
#         if tp + fp == 0: continue
#         precision = tp / (tp + fp)
#         recall = tp
#         precisionVec.append(precision)
#         recallVec.append(recall)
#         auprBase += (recallTemp-recall)*precision
#         recallTemp = recall
#     auprBase += recall * precision
#     #print(recall, precision)

#     # calculate our algorithm
# #     T = 1000
#     inddistr = np.loadtxt(PATH_CONFIDENCE_OUR_IN, delimiter=',')
#     outdistr = np.loadtxt(PATH_CONFIDENCE_OUR_OUT, delimiter=',')
# #     if name == "CIFAR-10": 
# #       start = 0.1
# #       end = 0.12 
# #     if name == "CIFAR-100": 
# #       start = 0.01
# #       end = 0.0104   
#     start = 0.001
#     end = 0.0011
#     gap = (end- start)/100000
#     #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
#     Y1 = outdistr[:, 2]
#     X1 = indistr[:, 2]
#     auprNew = 0.0
#     recallTemp = 1.0
#     for delta in np.arange(start, end, gap):
#         tp = np.sum(np.sum(X1 >= delta)) / np.float(len(X1))
#         fp = np.sum(np.sum(Y1 >= delta)) / np.float(len(Y1))
#         if tp + fp == 0: continue
#         precision = tp / (tp + fp)
#         recall = tp
#         #precisionVec.append(precision)
#         #recallVec.append(recall)
#         auprNew += (recallTemp-recall)*precision
#         recallTemp = recall
#     auprNew += recall * precision
#     return auprBase, auprNew

# def auprOut(name):
#     #calculate the AUPR
#     # calculate baseline
# #     T = 1
#     indistr = np.loadtxt(PATH_CONFIDENCE_BASE_IN, delimiter=',')
#     outdistr = np.loadtxt(PATH_CONFIDENCE_BASE_OUT, delimiter=',')
# #     if name == "CIFAR-10": 
# #       start = 0.1
# #       end = 1 
# #     if name == "CIFAR-100": 
#     start = 0.001
#     end = 1   
#     gap = (end- start)/100000
#     Y1 = outdistr[:, 2]
#     X1 = indistr[:, 2]
#     auprBase = 0.0
#     recallTemp = 1.0
#     for delta in np.arange(end, start, -gap):
#         fp = np.sum(np.sum(X1 < delta)) / np.float(len(X1))
#         tp = np.sum(np.sum(Y1 < delta)) / np.float(len(Y1))
#         if tp + fp == 0: break
#         precision = tp / (tp + fp)
#         recall = tp
#         auprBase += (recallTemp-recall)*precision
#         recallTemp = recall
#     auprBase += recall * precision
        
    
#     # calculate our algorithm
# #     T = 1000
#     indistr = np.loadtxt(PATH_CONFIDENCE_OUR_IN, delimiter=',')
#     outdistr = np.loadtxt(PATH_CONFIDENCE_OUR_OUT, delimiter=',')
# #     if name == "CIFAR-10": 
# #       start = 0.1
# #       end = 0.12 
# #     if name == "CIFAR-100": 
# #       start = 0.01
# #       end = 0.0104
#     start = 0.001
#     end = 0.0011
#     gap = (end- start)/100000
#     #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
#     Y1 = outdistr[:, 2]
#     X1 = indistr[:, 2]
#     auprNew = 0.0
#     recallTemp = 1.0
#     for delta in np.arange(end, start, -gap):
#         fp = np.sum(np.sum(X1 < delta)) / np.float(len(X1))
#         tp = np.sum(np.sum(Y1 < delta)) / np.float(len(Y1))
#         if tp + fp == 0: break
#         precision = tp / (tp + fp)
#         recall = tp
#         auprNew += (recallTemp-recall)*precision
#         recallTemp = recall
#     auprNew += recall * precision
#     return auprBase, auprNew



# def detection(name):
#     #calculate the minimum detection error
#     # calculate baseline
# #     T = 1
#     indistr = np.loadtxt(PATH_CONFIDENCE_BASE_IN, delimiter=',')
#     outdistr = np.loadtxt(PATH_CONFIDENCE_BASE_OUT, delimiter=',')
# #     if name == "CIFAR-10": 
# #       start = 0.1
# #       end = 1 
# #     if name == "CIFAR-100": 
#     start = 0.001
#     end = 1    
#     gap = (end- start)/100000
#     #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
#     Y1 = outdistr[:, 2]
#     X1 = indistr[:, 2]
#     errorBase = 1.0
#     for delta in np.arange(start, end, gap):
#         tpr = np.sum(np.sum(X1 < delta)) / np.float(len(X1))
#         error2 = np.sum(np.sum(Y1 > delta)) / np.float(len(Y1))
#         errorBase = np.minimum(errorBase, (tpr+error2)/2.0)

#     # calculate our algorithm
# #     T = 1000
#     indistr = np.loadtxt(PATH_CONFIDENCE_OUR_IN, delimiter=',')
#     outdistr = np.loadtxt(PATH_CONFIDENCE_OUR_OUT, delimiter=',')
# #     if name == "CIFAR-10": 
# #       start = 0.1
# #       end = 0.12 
# #     if name == "CIFAR-100": 
# #       start = 0.01
# #       end = 0.0104 
#     start = 0.001
#     end = 0.0011
#     gap = (end- start)/100000
#     #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
#     Y1 = outdistr[:, 2]
#     X1 = indistr[:, 2]
#     errorNew = 1.0
#     for delta in np.arange(start, end, gap):
#         tpr = np.sum(np.sum(X1 < delta)) / np.float(len(X1))
#         error2 = np.sum(np.sum(Y1 > delta)) / np.float(len(Y1))
#         errorNew = np.minimum(errorNew, (tpr+error2)/2.0)
            
#     return errorBase, errorNew


# def metric(nn):
#     indis = "Imagenet-1k"
#     fprBase, fprNew = tpr95(indis)
#     errorBase, errorNew = detection(indis)
#     aurocBase, aurocNew = auroc(indis)
#     auprinBase, auprinNew = auprIn(indis)
#     auproutBase, auproutNew = auprOut(indis)
#     print("{:31}{:>22}".format("Neural network architecture:", nn))
#     print("{:31}{:>22}".format("In-distribution dataset:", indis))
#     print("{:31}{:>22}".format("Out-of-distribution dataset:", "IMAGENET-O"))
#     print("")
#     print("{:>34}{:>19}".format("Baseline", "Our Method"))
#     print("{:20}{:13.1f}%{:>18.1f}% ".format("FPR at TPR 95%:",fprBase*100, fprNew*100))
#     print("{:20}{:13.1f}%{:>18.1f}%".format("Detection error:",errorBase*100, errorNew*100))
#     print("{:20}{:13.1f}%{:>18.1f}%".format("AUROC:",aurocBase*100, aurocNew*100))
#     print("{:20}{:13.1f}%{:>18.1f}%".format("AUPR In:",auprinBase*100, auprinNew*100))
#     print("{:20}{:13.1f}%{:>18.1f}%".format("AUPR Out:",auproutBase*100, auproutNew*100))

In [5]:
import numpy as np
import sklearn.metrics as sk
recall_level_default = 0.95

def stable_cumsum(arr, rtol=1e-05, atol=1e-08):
    """Use high precision for cumsum and check that final value matches sum
    Parameters
    ----------
    arr : array-like
        To be cumulatively summed as flat
    rtol : float
        Relative tolerance, see ``np.allclose``
    atol : float
        Absolute tolerance, see ``np.allclose``

    Credits: https://github.com/wetliu/energy_ood/blob/7d31247d980b5337c519a61402d35141f5ec6289/utils/display_results.py#L7
    """
    out = np.cumsum(arr, dtype=np.float64)
    expected = np.sum(arr, dtype=np.float64)
    if not np.allclose(out[-1], expected, rtol=rtol, atol=atol):
        raise RuntimeError('cumsum was found to be unstable: '
                           'its last element does not correspond to sum')
    return out

def calib_err(confidence, correct, p='2', beta=100):
    # beta is target bin size
    idxs = np.argsort(confidence)
    confidence = confidence[idxs]
    correct = correct[idxs]
    bins = [[i * beta, (i + 1) * beta] for i in range(len(confidence) // beta)]
    bins[-1] = [bins[-1][0], len(confidence)]

    cerr = 0
    total_examples = len(confidence)
    for i in range(len(bins) - 1):
        bin_confidence = confidence[bins[i][0]:bins[i][1]]
        bin_correct = correct[bins[i][0]:bins[i][1]]
        num_examples_in_bin = len(bin_confidence)

        if num_examples_in_bin > 0:
            difference = np.abs(np.nanmean(bin_confidence) - np.nanmean(bin_correct))

            if p == '2':
                cerr += num_examples_in_bin / total_examples * np.square(difference)
            elif p == '1':
                cerr += num_examples_in_bin / total_examples * difference
            elif p == 'infty' or p == 'infinity' or p == 'max':
                cerr = np.maximum(cerr, difference)
            else:
                assert False, "p must be '1', '2', or 'infty'"

    if p == '2':
        cerr = np.sqrt(cerr)

    return cerr


def aurra(confidence, correct):
    conf_ranks = np.argsort(confidence)[::-1]  # indices from greatest to least confidence
    rra_curve = np.cumsum(np.asarray(correct)[conf_ranks])
    rra_curve = rra_curve / np.arange(1, len(rra_curve) + 1)  # accuracy at each response rate
    return np.mean(rra_curve)


def soft_f1(confidence, correct):
    wrong = 1 - correct

    # # the incorrectly classified samples are our interest
    # # so they make the positive class
    # tp_soft = np.sum((1 - confidence) * wrong)
    # fp_soft = np.sum((1 - confidence) * correct)
    # fn_soft = np.sum(confidence * wrong)

    # return 2 * tp_soft / (2 * tp_soft + fn_soft + fp_soft)
    return 2 * ((1 - confidence) * wrong).sum()/(1 - confidence + wrong).sum()


def tune_temp(logits, labels, binary_search=True, lower=0.2, upper=5.0, eps=0.0001):
    logits = np.array(logits)

    if binary_search:
        import torch
        import torch.nn.functional as F

        logits = torch.FloatTensor(logits)
        labels = torch.LongTensor(labels)
        t_guess = torch.FloatTensor([0.5*(lower + upper)]).requires_grad_()

        while upper - lower > eps:
            if torch.autograd.grad(F.cross_entropy(logits / t_guess, labels), t_guess)[0] > 0:
                upper = 0.5 * (lower + upper)
            else:
                lower = 0.5 * (lower + upper)
            t_guess = t_guess * 0 + 0.5 * (lower + upper)

        t = min([lower, 0.5 * (lower + upper), upper], key=lambda x: float(F.cross_entropy(logits / x, labels)))
    else:
        import cvxpy as cx

        set_size = np.array(logits).shape[0]

        t = cx.Variable()

        expr = sum((cx.Minimize(cx.log_sum_exp(logits[i, :] * t) - logits[i, labels[i]] * t)
                    for i in range(set_size)))
        p = cx.Problem(expr, [lower <= t, t <= upper])

        p.solve()   # p.solve(solver=cx.SCS)
        t = 1 / t.value

    return t


def print_measures(rms, aurra_metric, mad, sf1, method_name='Baseline'):
    print('\t\t\t\t\t\t\t' + method_name)
    print('RMS Calib Error (%): \t\t{:.2f}'.format(100 * rms))
    print('AURRA (%): \t\t\t{:.2f}'.format(100 * aurra))
    # print('MAD Calib Error (%): \t\t{:.2f}'.format(100 * mad))
    # print('Soft F1 Score (%):   \t\t{:.2f}'.format(100 * sf1))


def show_calibration_results(confidence, correct, method_name='Baseline'):

    print('\t\t\t\t' + method_name)
    print('RMS Calib Error (%): \t\t{:.2f}'.format(
        100 * calib_err(confidence, correct, p='2')))

    print('AURRA (%): \t\t\t{:.2f}'.format(
        100 * aurra(confidence, correct)))

    # print('MAD Calib Error (%): \t\t{:.2f}'.format(
    #     100 * calib_err(confidence, correct, p='1')))

    # print('Soft F1-Score (%): \t\t{:.2f}'.format(
    #     100 * soft_f1(confidence, correct)))

def fpr_and_fdr_at_recall(y_true, y_score, recall_level=recall_level_default, pos_label=None):
    classes = np.unique(y_true)
    if (pos_label is None and
            not (np.array_equal(classes, [0, 1]) or
                     np.array_equal(classes, [-1, 1]) or
                     np.array_equal(classes, [0]) or
                     np.array_equal(classes, [-1]) or
                     np.array_equal(classes, [1]))):
        raise ValueError("Data is not binary and pos_label is not specified")
    elif pos_label is None:
        pos_label = 1.

    # make y_true a boolean vector
    y_true = (y_true == pos_label)

    # sort scores and corresponding truth values
    desc_score_indices = np.argsort(y_score, kind="mergesort")[::-1]
    y_score = y_score[desc_score_indices]
    y_true = y_true[desc_score_indices]

    # y_score typically has many tied values. Here we extract
    # the indices associated with the distinct values. We also
    # concatenate a value for the end of the curve.
    distinct_value_indices = np.where(np.diff(y_score))[0]
    threshold_idxs = np.r_[distinct_value_indices, y_true.size - 1]

    # accumulate the true positives with decreasing threshold
    tps = stable_cumsum(y_true)[threshold_idxs]
    fps = 1 + threshold_idxs - tps      # add one because of zero-based indexing

    thresholds = y_score[threshold_idxs]

    recall = tps / tps[-1]

    last_ind = tps.searchsorted(tps[-1])
    sl = slice(last_ind, None, -1)      # [last_ind::-1]
    recall, fps, tps, thresholds = np.r_[recall[sl], 1], np.r_[fps[sl], 0], np.r_[tps[sl], 0], thresholds[sl]

    cutoff = np.argmin(np.abs(recall - recall_level))

    return fps[cutoff] / (np.sum(np.logical_not(y_true)))   # , fps[cutoff]/(fps[cutoff] + tps[cutoff])

def get_measures(_pos, _neg, recall_level=recall_level_default):
    pos = np.array(_pos[:]).reshape((-1, 1))
    neg = np.array(_neg[:]).reshape((-1, 1))
    examples = np.squeeze(np.vstack((pos, neg)))
    labels = np.zeros(len(examples), dtype=np.int32)
    labels[:len(pos)] += 1

    auroc = sk.roc_auc_score(labels, examples)
    aupr = sk.average_precision_score(labels, examples)
    fpr = fpr_and_fdr_at_recall(labels, examples, recall_level)

    return auroc, aupr, fpr


def print_measures_old(auroc, aupr, fpr, method_name='Ours', recall_level=recall_level_default):
    print('\t\t\t' + method_name)
    print('FPR{:d}:\t{:.2f}'.format(int(100 * recall_level), 100 * fpr))
    print('AUROC: \t{:.2f}'.format(100 * auroc))
    print('AUPR:  \t{:.2f}'.format(100 * aupr))


def print_measures_with_std(aurocs, auprs, fprs, method_name='Ours', recall_level=recall_level_default):
    print('\t\t\t' + method_name)
    print('FPR{:d}:\t{:.2f}\t+/- {:.2f}'.format(int(100 * recall_level), 100 * np.mean(fprs), 100 * np.std(fprs)))
    print('AUROC: \t{:.2f}\t+/- {:.2f}'.format(100 * np.mean(aurocs), 100 * np.std(aurocs)))
    print('AUPR:  \t{:.2f}\t+/- {:.2f}'.format(100 * np.mean(auprs), 100 * np.std(auprs)))


def get_and_print_results(out_score, in_score, num_to_avg=1):

    aurocs, auprs, fprs = [], [], []
    #for _ in range(num_to_avg):
    #    out_score = get_ood_scores(ood_loader)
    measures = get_measures(out_score, in_score)
    aurocs.append(measures[0]); auprs.append(measures[1]); fprs.append(measures[2])

    auroc = np.mean(aurocs); aupr = np.mean(auprs); fpr = np.mean(fprs)
    #auroc_list.append(auroc); aupr_list.append(aupr); fpr_list.append(fpr)

    #if num_to_avg >= 5:
    #    print_measures_with_std(aurocs, auprs, fprs, method_name='Ours')
    #else:
    #    print_measures(auroc, aupr, fpr, method_name='Ours')
    return auroc, aupr, fpr 

In [6]:
to_np = lambda x: x.data.to('cpu').numpy()

def testData(net, criterion, CUDA_DEVICE, testloader10, testloader, noiseMagnitude1, temper):
    t0 = time.time()
    f1 = open(PATH_CONFIDENCE_BASE_IN, 'w+')
    f2 = open(PATH_CONFIDENCE_BASE_OUT, 'w+')
    g1 = open(PATH_CONFIDENCE_OUR_IN, 'w+')
    g2 = open(PATH_CONFIDENCE_OUR_OUT, 'w+')
    N = 10000
    print("Processing in-distribution images")
########################################In-distribution###########################################
    for j, data in enumerate(testloader10):
#         if j<1000: continue
        images, _ = data
        
        inputs = Variable(images.cuda(CUDA_DEVICE), requires_grad = True)
        outputs = net(inputs)
        

        # Calculating the confidence of the output, no perturbation added here, no temperature scaling used
        nnOutputs = outputs.data.cpu()
        nnOutputs = nnOutputs.numpy()
        nnOutputs = nnOutputs[0]
        nnOutputs = nnOutputs - np.max(nnOutputs)
        nnOutputs = np.exp(nnOutputs)/np.sum(np.exp(nnOutputs))
#         print(temper, noiseMagnitude1, np.max(nnOutputs))
        f1.write("{}, {}, {}\n".format(temper, noiseMagnitude1, np.max(nnOutputs)))
        
        # Using temperature scaling
        outputs = outputs / temper
	
        # Calculating the perturbation we need to add, that is,
        # the sign of gradient of cross entropy loss w.r.t. input
        maxIndexTemp = np.argmax(nnOutputs)
        labels = Variable(torch.LongTensor([maxIndexTemp]).cuda(CUDA_DEVICE))
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Normalizing the gradient to binary in {0, 1}
        gradient =  torch.ge(inputs.grad.data, 0)
        gradient = (gradient.float() - 0.5) * 2
        # Normalizing the gradient to the same space of image  # std: [0.229, 0.224, 0.225]
        gradient[0][0] = (gradient[0][0] )/0.229
        gradient[0][1] = (gradient[0][1] )/0.224
        gradient[0][2] = (gradient[0][2])/0.225
        # Adding small perturbations to images
        tempInputs = torch.add(inputs.data,  -noiseMagnitude1, gradient)
        outputs = net(Variable(tempInputs))
        outputs = outputs / temper
        # Calculating the confidence after adding perturbations
        nnOutputs = outputs.data.cpu()
        nnOutputs = nnOutputs.numpy()
        nnOutputs = nnOutputs[0]
        nnOutputs = nnOutputs - np.max(nnOutputs)
        nnOutputs = np.exp(nnOutputs)/np.sum(np.exp(nnOutputs))
        g1.write("{}, {}, {}\n".format(temper, noiseMagnitude1, np.max(nnOutputs)))
        if j % 100 == 99:
#             print("{:4}/{:4} images processed, {:.1f} seconds used.".format(j+1-1000, N-1000, time.time()-t0))
            t0 = time.time()
        
        if j == N - 1: break


    t0 = time.time()
    print("Processing out-of-distribution images")
###################################Out-of-Distributions#####################################
    for j, data in enumerate(testloader):
#         if j<1000: continue
        images, _ = data
    
        inputs = Variable(images.cuda(CUDA_DEVICE), requires_grad = True)
        outputs = net(inputs)


        # Calculating the confidence of the output, no perturbation added here
        nnOutputs = outputs.data.cpu()
        nnOutputs = nnOutputs.numpy()
        nnOutputs = nnOutputs[0]
        nnOutputs = nnOutputs - np.max(nnOutputs)
        nnOutputs = np.exp(nnOutputs)/np.sum(np.exp(nnOutputs))
#         print(temper, noiseMagnitude1, np.max(nnOutputs))
        f2.write("{}, {}, {}\n".format(temper, noiseMagnitude1, np.max(nnOutputs)))
        
        # Using temperature scaling
        outputs = outputs / temper
  
  
        # Calculating the perturbation we need to add, that is,
        # the sign of gradient of cross entropy loss w.r.t. input
        maxIndexTemp = np.argmax(nnOutputs)
        labels = Variable(torch.LongTensor([maxIndexTemp]).cuda(CUDA_DEVICE))
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Normalizing the gradient to binary in {0, 1}
        gradient =  (torch.ge(inputs.grad.data, 0))
        gradient = (gradient.float() - 0.5) * 2
        # Normalizing the gradient to the same space of image # std: [0.229, 0.224, 0.225]
        gradient[0][0] = (gradient[0][0] )/0.229
        gradient[0][1] = (gradient[0][1] )/0.224
        gradient[0][2] = (gradient[0][2])/0.225
        # Adding small perturbations to images
        tempInputs = torch.add(inputs.data,  -noiseMagnitude1, gradient)
        outputs = net(Variable(tempInputs))
        outputs = outputs / temper
        # Calculating the confidence after adding perturbations
        nnOutputs = outputs.data.cpu()
        nnOutputs = nnOutputs.numpy()
        nnOutputs = nnOutputs[0]
        nnOutputs = nnOutputs - np.max(nnOutputs)
        nnOutputs = np.exp(nnOutputs)/np.sum(np.exp(nnOutputs))
        g2.write("{}, {}, {}\n".format(temper, noiseMagnitude1, np.max(nnOutputs)))
        if j % 100 == 99:
#             print("{:4}/{:4} images processed, {:.1f} seconds used.".format(j+1-1000, N-1000, time.time()-t0))
            t0 = time.time()

        if j== N-1: break

In [ ]:
PATH_TO_IMAGENET_O = "/scratch/as14770/CV-FinalProject/DATA/imagenet-o"
PATH_TO_IMAGENET_VAL = "/scratch/as14770/CV-FinalProject/DATA/imagenet1k-val"
TORCH_HOME_DIR = "/scratch/as14770/CV-FinalProject/.cvfpmodels/"

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

test_transform = trn.Compose(
    [trn.Resize(256), trn.CenterCrop(224), trn.ToTensor(), trn.Normalize(mean, std)])

noes = dset.ImageFolder(root=PATH_TO_IMAGENET_O, transform=test_transform)
noe_loader = torch.utils.data.DataLoader(noes, batch_size=1, shuffle=False,
                                         num_workers=4, pin_memory=True)

imagenet_o_folder = "/scratch/as14770/CV-FinalProject/imagenet-o-symlink/"

def create_symlinks_to_imagenet(imagenet_folder, folder_to_scan):
    if not os.path.exists(imagenet_folder):
        os.makedirs(imagenet_folder)
        folders_of_interest = os.listdir(folder_to_scan)
        path_prefix = PATH_TO_IMAGENET_VAL 
        for folder in folders_of_interest:
            os.symlink(path_prefix + folder, imagenet_folder+folder, target_is_directory=True)

create_symlinks_to_imagenet(imagenet_o_folder, PATH_TO_IMAGENET_O)

val_examples_imagenet_o = dset.ImageFolder(root=imagenet_o_folder, transform=test_transform)
val_loader_imagenet_o = torch.utils.data.DataLoader(val_examples_imagenet_o, batch_size=1, shuffle=False,
                                         num_workers=4, pin_memory=True)

val_imagenet = dset.ImageNet(root=PATH_TO_IMAGENET_VAL, split = 'val', transform=test_transform)
val_imagenet_loader = torch.utils.data.DataLoader(val_imagenet, batch_size=1, shuffle=False,
                                         num_workers=4, pin_memory=True)

concat = lambda x: np.concatenate(x, axis=0)
to_np = lambda x: x.data.to('cpu').numpy()

os.environ["TORCH_HOME"] = TORCH_HOME_DIR
#print(os.environ["TORCH_HOME"])
torch.hub.set_dir(TORCH_HOME_DIR)
models_to_test = [
    ("pytorch/vision", "alexnet"),
    ("pytorch/vision", "squeezenet1_1"),
    ("pytorch/vision", "vgg16"),
    ("pytorch/vision", "vgg19"),
    ('pytorch/vision', "vgg19_bn"),
    ('pytorch/vision', "densenet121"),
    ('pytorch/vision', "resnet50"),
    ('pytorch/vision', "resnet101"),
    ('pytorch/vision', "resnet152"),
    ('pytorch/vision', "wide_resnet50_2"),
    ('pytorch/vision', "resnext101_32x8d"),
    ('pytorch/vision', "resnext50_32x4d"),
    ('facebookresearch/WSL-Images', "resnext101_32x8d_wsl"),
    ('facebookresearch/WSL-Images', "resnext101_32x16d_wsl"),
    ('facebookresearch/WSL-Images', "resnext101_32x32d_wsl"),
    ("pretrained", "dpn68"),
    ("pretrained", "dpn98"),
    ("pretrained", "se_resnet101"),
    ("pretrained", "se_resnet152"),
    ("pretrained", "resnext101_32x4d"),
    ("pretrained", "se_resnext101_32x4d"),
    ("facebookresearch/deit:main", "deit_base_patch16_224"),
    ("facebookresearch/deit:main", "deit_small_patch16_224"),
    ("facebookresearch/deit:main", "deit_tiny_patch16_224"),
]

for net_params in models_to_test:
    if net_params[0] == "pytorch/vision":
        net = torch.hub.load('pytorch/vision:v0.10.0', net_params[1], pretrained=True)
#         net = models.alexnet(pretrained=True)
    elif "facebookresearch/deit" in net_params[0]:
        net = torch.hub.load(net_params[0], net_params[1], pretrained=True)
    elif net_params[0] == "pretrained":
        net = pretrainedmodels.__dict__[net_params[1]](num_classes=1000, pretrained='imagenet')
    else:
        net = torch.hub.load('/scratch/as14770/CV-FinalProject/facebookresearch_WSL-Images_master', net_params[1], source='local')
    print(net_params[1], '\n')
    net.cuda()
    net.eval()
    testData(net, criterion, CUDA_DEVICE, val_loader_imagenet_o, noe_loader, epsilon, temperature) 
    # Get Baseline Results
    indistr = np.loadtxt(PATH_CONFIDENCE_BASE_IN, delimiter=',')
    outdistr = np.loadtxt(PATH_CONFIDENCE_BASE_OUT, delimiter=',')
#     start = 0.001
#     end = 1    
#     gap = (end- start)/100000
    Y1 = outdistr[:, 2]
#     Y1 = -Y1
    X1 = indistr[:, 2]
#     X1 = -X1
    aurocs, auprs, fprs = [], [], []
    measures = get_measures(Y1, X1)
    aurocs = measures[0]; auprs = measures[1]; fprs = measures[2]
    print_measures_old(aurocs, auprs, fprs, method_name='Baseline')
    # Get ODIN Results
    indistr = np.loadtxt(PATH_CONFIDENCE_OUR_IN, delimiter=',')
    outdistr = np.loadtxt(PATH_CONFIDENCE_OUR_OUT, delimiter=',')
#     start = 0.001 #To be tuned
#     end = 0.0011 # To be tuned
#     gap = (end- start)/100000
    Y1 = outdistr[:, 2]
#     Y1 = -Y1
    X1 = indistr[:, 2]
#     X1 = -X1
    aurocs, auprs, fprs = [], [], []
    measures = get_measures(Y1, X1)
    aurocs = measures[0]; auprs = measures[1]; fprs = measures[2]
    print_measures_old(aurocs, auprs, fprs, method_name='ODIN')
#     metric(net_params[1])
    print("\n\n\n")

Using cache found in /scratch/as14770/CV-FinalProject/.cvfpmodels/pytorch_vision_v0.10.0


alexnet 

Processing in-distribution images


/state/partition1/job-12686078/ipykernel_1478858/1984889901.py:47: UserWarning: This overload of add is deprecated:
	add(Tensor input, Number alpha, Tensor other, *, Tensor out)
Consider using one of the following signatures instead:
	add(Tensor input, Tensor other, *, Number alpha, Tensor out) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)
  tempInputs = torch.add(inputs.data,  -noiseMagnitude1, gradient)


Processing out-of-distribution images
			Baseline
FPR95:	88.01
AUROC: 	50.51
AUPR:  	15.33
			ODIN
FPR95:	89.14
AUROC: 	52.07
AUPR:  	15.96




squeezenet1_1 

Processing in-distribution images


Using cache found in /scratch/as14770/CV-FinalProject/.cvfpmodels/pytorch_vision_v0.10.0


...

vgg19 

Processing in-distribution images
Processing out-of-distribution images
			Baseline
FPR95:	90.44
AUROC: 	46.28
AUPR:  	14.08
			ODIN
FPR95:	91.12
AUROC: 	43.00
AUPR:  	13.34






Using cache found in /scratch/as14770/CV-FinalProject/.cvfpmodels/pytorch_vision_v0.10.0


vgg19_bn 

Processing in-distribution images
Processing out-of-distribution images
			Baseline
FPR95:	87.17
AUROC: 	46.88
AUPR:  	14.19
			ODIN
FPR95:	90.64
AUROC: 	42.71
AUPR:  	13.28






Using cache found in /scratch/as14770/CV-FinalProject/.cvfpmodels/pytorch_vision_v0.10.0


densenet121 

Processing in-distribution images
